#### Importações

In [242]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [243]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [244]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [245]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [246]:
original_x_gan.head(25)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,123.929000,22.307377,23.563036,99.000,99.000,107.500,9.975000e+01,1.160000e+02,1.170000e+02,142.500000,1.510000e+02,1.560000e+02,1.560000e+02
weight (t),18815.714286,18773.265300,5472.264453,5892.899690,10054.000,10057.000,16530.500,1.302300e+04,1.858400e+04,1.865250e+04,21182.000000,2.452300e+04,2.764700e+04,2.764400e+04
installation_date,38.857143,38.485400,5.639993,5.643418,31.000,31.000,34.500,3.200000e+01,4.100000e+01,3.900000e+01,42.500000,4.500000e+01,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.499800,0.786796,0.809238,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,7.324000,1.511858,1.499083,4.000,4.000,8.000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,16.705800,10.106575,9.775480,5.000,5.000,8.500,7.000000e+00,2.000000e+01,1.500000e+01,23.000000,2.700000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,137.210353,18.993420,19.000237,114.000,114.001,122.750,1.180570e+02,1.360000e+02,1.340400e+02,150.500000,1.570362e+02,1.660000e+02,1.659990e+02
distance_to_coast (km),218.571429,216.603500,66.271807,69.443709,120.000,120.000,180.000,1.290000e+02,2.400000e+02,2.580000e+02,264.000000,2.810000e+02,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.414100,0.534522,0.492591,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000006,0.000009,0.000009,0.000,0.000,0.000,4.729629e-11,7.800000e-08,3.755154e-08,0.000012,1.576528e-05,2.300000e-05,2.300000e-05


#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [247]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [248]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [249]:
# df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

In [250]:
# df_gan_limpo.shape

#### Removendo outliers com base na amplitude interquartil

In [251]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [252]:
#df_gan_limpo = remover_outliers_iqr(df_gan)

In [253]:
# df_gan.shape, df_gan_limpo.shape

In [254]:
#df_gan_limpo.duplicated().sum()

#### Comparando novamente as bases de dados

In [255]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [256]:
original_x_gan.head(26)

,mean_original,mean_gan,std_original,std_gan,min_original,min_gan,25%_original,25%_gan,50%_original,50%_gan,75%_original,75%_gan,max_original,max_gan
water_depth (m),124.428571,123.929000,22.307377,23.563036,99.000,99.000,107.500,9.975000e+01,1.160000e+02,1.170000e+02,142.500000,1.510000e+02,1.560000e+02,1.560000e+02
weight (t),18815.714286,18773.265300,5472.264453,5892.899690,10054.000,10057.000,16530.500,1.302300e+04,1.858400e+04,1.865250e+04,21182.000000,2.452300e+04,2.764700e+04,2.764400e+04
installation_date,38.857143,38.485400,5.639993,5.643418,31.000,31.000,34.500,3.200000e+01,4.100000e+01,3.900000e+01,42.500000,4.500000e+01,4.600000e+01,4.600000e+01
type_of_production (1 oil and gas; 2 oil; 3 gas),1.571429,1.499800,0.786796,0.809238,1.000,1.000,1.000,1.000000e+00,1.000000e+00,1.000000e+00,2.000000,2.000000e+00,3.000000e+00,3.000000e+00
number_of_legs,7.428571,7.324000,1.511858,1.499083,4.000,4.000,8.000,8.000000e+00,8.000000e+00,8.000000e+00,8.000000,8.000000e+00,8.000000e+00,8.000000e+00
number_of_piles,17.142857,16.705800,10.106575,9.775480,5.000,5.000,8.500,7.000000e+00,2.000000e+01,1.500000e+01,23.000000,2.700000e+01,3.200000e+01,3.100000e+01
height_of_jacket_or_sub-structure (m),137.500000,137.210353,18.993420,19.000237,114.000,114.001,122.750,1.180570e+02,1.360000e+02,1.340400e+02,150.500000,1.570362e+02,1.660000e+02,1.659990e+02
distance_to_coast (km),218.571429,216.603500,66.271807,69.443709,120.000,120.000,180.000,1.290000e+02,2.400000e+02,2.580000e+02,264.000000,2.810000e+02,2.820000e+02,2.820000e+02
risk_to_other_users-complete,0.428571,0.414100,0.534522,0.492591,0.000,0.000,0.000,0.000000e+00,0.000000e+00,0.000000e+00,1.000000,1.000000e+00,1.000000e+00,1.000000e+00
risk_to_other_users-partial,0.000007,0.000006,0.000009,0.000009,0.000,0.000,0.000,4.729629e-11,7.800000e-08,3.755154e-08,0.000012,1.576528e-05,2.300000e-05,2.300000e-05


In [257]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

,mean,std,min,25%,50%,75%,max
water_depth (m),4.995714e-01,-1.255660e+00,0.000,7.750000e+00,-1.000000e+00,-8.500000,0.000
weight (t),4.244899e+01,-4.206352e+02,-3.000,3.507500e+03,-6.850000e+01,-3341.000000,3.000
installation_date,3.717429e-01,-3.425192e-03,0.000,2.500000e+00,2.000000e+00,-2.500000,0.000
type_of_production (1 oil and gas; 2 oil; 3 gas),7.162857e-02,-2.244178e-02,0.000,0.000000e+00,0.000000e+00,0.000000,0.000
number_of_legs,1.045714e-01,1.277526e-02,0.000,0.000000e+00,0.000000e+00,0.000000,0.000
number_of_piles,4.370571e-01,3.310953e-01,0.000,1.500000e+00,5.000000e+00,-4.000000,1.000
height_of_jacket_or_sub-structure (m),2.896471e-01,-6.817295e-03,-0.001,4.693000e+00,1.960000e+00,-6.536250,0.001
distance_to_coast (km),1.967929e+00,-3.171902e+00,0.000,5.100000e+01,-1.800000e+01,-17.000000,0.000
risk_to_other_users-complete,1.447143e-02,4.193193e-02,0.000,0.000000e+00,0.000000e+00,0.000000,0.000
risk_to_other_users-partial,3.263062e-07,-1.587236e-07,0.000,-4.729629e-11,4.044846e-08,-0.000004,0.000


#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [258]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [259]:
df_original.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,impacts_of_option-partial,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,99,22000,36,1,8,8,114.0,269,1,0.000000e+00,0.290,0.12,1110100,817000,87000,58500,0.00,1.0,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
1,112,20364,41,1,8,9,123.0,259,1,0.000000e+00,0.320,0.16,1180500,895500,92000,64000,0.00,1.0,0.00,1.00,0.79,0.78,0.5,0.5,1.00,0.56
2,141,15561,46,2,8,26,147.0,120,0,2.300000e-05,0.025,0.01,297654,530148,24277,31064,0.66,1.0,0.25,1.00,1.00,0.94,1.0,1.0,0.53,1.00
3,156,27647,44,2,8,32,166.0,240,0,1.500000e-05,0.040,0.02,487750,570818,40416,45266,1.00,1.0,0.50,1.00,1.00,0.66,1.0,1.0,1.00,0.57
4,103,18584,33,1,8,20,122.5,230,0,7.800000e-08,0.090,0.06,733082,511765,59588,41170,0.00,1.0,0.39,0.15,0.00,0.00,0.0,0.0,1.00,0.73


In [260]:
df_gan.head()

,water_depth (m),weight (t),installation_date,type_of_production (1 oil and gas; 2 oil; 3 gas),number_of_legs,number_of_piles,height_of_jacket_or_sub-structure (m),distance_to_coast (km),risk_to_other_users-complete,risk_to_other_users-partial,risk_to_personnel-complete,risk_to_personnel-partial,energy_consumption-complete (GJ),energy_consumption-partial (GJ),emissions-complete (t),emissions-partial (t),impacts_of_option-complete,impacts_of_option-partial,technical_feasibility_or_challenge-complete,technical_feasibility_or_challenge-partial,commercial_impact_on_fisheries-complete,commercial_impact_on_fisheries-partial,wider_community_impact-complete,wider_community_impact-partial,total_removal_cost-complete,total_removal_cost-partial
0,143,14058,41,1,4,5,152.271,239,1,2.418965e-09,0.315,0.159,842794,757692,54029,51206,0.000,1,0.000,1.000,0.618,0.812,0.241,0.460,0.978,0.422
1,155,27600,45,3,8,28,164.516,143,0,4.048576e-06,0.026,0.011,535099,634491,41327,63282,0.577,0,0.500,0.801,0.233,0.001,0.489,0.564,0.983,0.410
2,101,21361,42,1,8,9,115.138,150,1,6.512493e-09,0.318,0.159,1137364,874276,90522,61405,0.000,1,0.000,0.998,0.261,0.773,0.235,0.097,0.857,0.921
3,155,26364,45,3,8,28,165.716,130,0,2.280883e-05,0.025,0.011,305580,495571,24573,38002,0.992,1,0.500,0.999,0.995,0.547,0.981,0.981,0.862,0.444
4,115,20751,39,1,8,15,117.076,141,0,3.468668e-08,0.252,0.100,865234,743272,77186,48927,0.000,1,0.307,0.405,0.012,0.019,0.085,0.065,0.908,0.838


In [261]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# # Criar o metadata
# metadata = SingleTableMetadata()
# metadata.detect_from_dataframe(data=A)
# metadata_dict = metadata.to_dict()
metadata_dict = {
    "columns": {
        "water_depth (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "weight (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "installation_date": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "type_of_production (1 oil and gas; 2 oil; 3 gas)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_legs": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_piles": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "height_of_jacket_or_sub-structure (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "distance_to_coast (km)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "risk_to_other_users-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_other_users-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "energy_consumption-complete (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "energy_consumption-partial (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-complete (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "emissions-partial (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "numerical"
        },
        "impacts_of_option-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "impacts_of_option-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        }
    },
    "primary_key": "water_depth (m)",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 26/26 [00:00<00:00, 441.65it/s]|
Column Shapes Score: 77.93%

(2/2) Evaluating Column Pair Trends: |██████████| 325/325 [00:02<00:00, 160.68it/s]|
Column Pair Trends Score: 93.16%

Overall Score (Average): 85.55%



In [262]:
report.get_score()

0.8554837183348845

In [263]:
report.get_properties()

,Property,Score
0,Column Shapes,0.779321
1,Column Pair Trends,0.931647


In [264]:
report.get_details(property_name='Column Shapes')

,Column,Metric,Score
0,water_depth (m),KSComplement,0.787757
1,weight (t),KSComplement,0.768857
2,installation_date,KSComplement,0.857543
3,type_of_production (1 oil and gas; 2 oil; 3 gas),KSComplement,0.868829
4,number_of_legs,KSComplement,0.973857
5,number_of_piles,KSComplement,0.835671
6,height_of_jacket_or_sub-structure (m),KSComplement,0.790257
7,distance_to_coast (km),KSComplement,0.687457
8,risk_to_other_users-complete,KSComplement,0.985529
9,risk_to_other_users-partial,KSComplement,0.691829


In [265]:
report.get_details(property_name='Column Pair Trends')

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,water_depth (m),weight (t),CorrelationSimilarity,0.952765,0.242657,0.148188
1,water_depth (m),installation_date,CorrelationSimilarity,0.983910,0.742407,0.774587
2,water_depth (m),type_of_production (1 oil and gas; 2 oil; 3 gas),CorrelationSimilarity,0.943402,0.828860,0.715664
3,water_depth (m),number_of_legs,CorrelationSimilarity,0.987891,0.166611,0.142392
4,water_depth (m),number_of_piles,CorrelationSimilarity,0.828663,0.773687,0.431013
5,water_depth (m),height_of_jacket_or_sub-structure (m),CorrelationSimilarity,0.861735,0.980269,0.703738
6,water_depth (m),distance_to_coast (km),CorrelationSimilarity,0.946050,-0.592407,-0.700308
7,water_depth (m),risk_to_other_users-complete,CorrelationSimilarity,0.991856,-0.646966,-0.630678
8,water_depth (m),risk_to_other_users-partial,CorrelationSimilarity,0.944323,0.824868,0.713514
9,water_depth (m),risk_to_personnel-complete,CorrelationSimilarity,0.868801,-0.649726,-0.387328
